In [1]:
from torchvision import datasets
from torchvision import transforms
import torch

data_folder = "data/FMNIST"
train = datasets.FashionMNIST(root=data_folder,
                              train=True,
                              transform=transforms.ToTensor(),
                              target_transform=None,
                              download = True)
test = datasets.FashionMNIST(root=data_folder,
                             train = False,
                             download=True,
                             transform=transforms.ToTensor())

100%|██████████| 26.4M/26.4M [00:01<00:00, 17.6MB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 302kB/s]
100%|██████████| 4.42M/4.42M [00:00<00:00, 5.58MB/s]
100%|██████████| 5.15k/5.15k [00:00<00:00, 11.5MB/s]


In [2]:
from torch.utils.data import DataLoader
BATCH_SIZE = 32

train_dataloader = DataLoader(dataset=train,
                              batch_size=BATCH_SIZE,
                              shuffle =True)

test_dataloader = DataLoader(dataset=test,
                             batch_size=BATCH_SIZE,
                             shuffle=False)

len(train_dataloader), len(test_dataloader)

(1875, 313)

In [3]:
import torch.nn as nn
class FashionMNISTV1(nn.Module):
  def __init__(self, input_features=28*28, output_features=10, hidden_layers=1000):
    super().__init__()
    self.layer = nn.Sequential(
        nn.Flatten(),
        nn.Linear(input_features, hidden_layers),
        nn.ReLU(),
        nn.Linear(hidden_layers, output_features)
    )
  def forward(self, x):
    return self.layer(x)

In [4]:
model = FashionMNISTV1()

In [6]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

In [7]:
def accuracy(y_true, y_pred):
  correct = torch.eq(y_true, y_pred).sum().item()
  acc = (correct / len(y_pred)) * 100
  return acc

In [9]:
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,
    factor=0.5,
    patience=0,
    threshold=0.001,
    min_lr=1e-5,
    threshold_mode="abs",
)

epochs = 10
from tqdm.auto import tqdm
for epoch in tqdm(range(epochs)):
  print(f"currently in epoch: {epoch} of {epochs}")
  train_loss = 0
  train_acc = 0
  for batch, (X, y) in enumerate(train_dataloader):
    model.train()
    y_pred = model(X)
    loss = loss_fn(y_pred, y)
    train_loss += loss
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    train_acc += accuracy(y, y_pred.argmax(dim=1))
  train_loss /= len(train_dataloader)
  train_acc /= len(train_dataloader)


  test_loss, test_acc = 0, 0
  model.eval()
  with torch.inference_mode():
    for X_test, y_test in test_dataloader:
      test_pred = model(X_test)
      test_loss += loss_fn(test_pred, y_test)
      test_acc = accuracy(y_test,test_pred.argmax(dim=1))
  print(f"train los: {train_loss}\n train acc: {train_acc}\n test loss: {test_loss}\n test_acc: {test_acc}")

  0%|          | 0/10 [00:00<?, ?it/s]

currently in epoch: 0 of 10
train los: 0.541775107383728
 train acc: 80.935
 test loss: 156.13511657714844
 test_acc: 100.0
currently in epoch: 1 of 10
train los: 0.438370019197464
 train acc: 84.285
 test loss: 147.45286560058594
 test_acc: 87.5
currently in epoch: 2 of 10
train los: 0.42111408710479736
 train acc: 84.79
 test loss: 142.2931365966797
 test_acc: 87.5
currently in epoch: 3 of 10
train los: 0.400332510471344
 train acc: 85.58833333333334
 test loss: 142.93138122558594
 test_acc: 93.75
currently in epoch: 4 of 10
train los: 0.3902650475502014
 train acc: 85.79833333333333
 test loss: 140.77880859375
 test_acc: 100.0
currently in epoch: 5 of 10
train los: 0.37874436378479004
 train acc: 86.135
 test loss: 138.1396942138672
 test_acc: 87.5
currently in epoch: 6 of 10
train los: 0.379176527261734
 train acc: 86.34333333333333
 test loss: 141.56655883789062
 test_acc: 81.25
currently in epoch: 7 of 10
train los: 0.36901718378067017
 train acc: 86.58
 test loss: 134.9786376953